### 1.Prepare to run

#### 1.1 Import modules 

In [1]:
# import existing ones
import csv
import regex as re
from collections import defaultdict, Counter
import en_coref_md
import glob

In [2]:
# import modules we created
import utils

#### 1.2 Set data directories

In [3]:
# Important: set which partition to work with!
which_partition='partial'

In [4]:
# SemEval input files (annotations and text)

# annotation data
se_annotation_dir='../data/input/%s/annotation' % which_partition
se_annotation_file='%s/participants_input.p' % se_annotation_dir

# text documents
se_partial_text_file='../data/input/partial/text/docs.p'
se_all_documents_path='../data/input/full/text'

# SemEval output paths
se_output_dir='extracted_data/%s' % which_partition
se_output_file='%s/extracted_data.p' % se_output_dir

In [5]:
# GVDB data file
gvdb_articles_file = '../../gvdb-aggregated-db/Articles-with-extracted-info.tsv'

#### 1.3 Load lexicons

In [6]:
emiel_resources_file='../resources/emiel.json'
emiel_resources=utils.load_json(emiel_resources_file)

students_resources_file='../resources/students.json'
students_resources=utils.load_json(students_resources_file)

genders_resources_file='../resources/genders.json'
genders=utils.load_json(genders_resources_file)

#### 1.4 Load coreference model

In [7]:
nlp = en_coref_md.load()

### 2. Generate keywords

#### 2.1 Define mapping of properties (lower-to-capitalletter)

In [8]:
map_properties ={
                    'age': 'Age', 
                    'ethnicity': 'Ethnicity',
                    #'occupation': 'Occupation',
                    'causeofdeath': 'CauseOfDeath',
                    'religion': 'Religion',
                    'educationlevel': 'EducationLevel',
                    'pastconviction': 'PastConviction',
                    'residence': 'Residence',
                    'birthplace': 'BirthPlace',
                    'gender': 'Gender'
                }

In [9]:
se_attributes=list(map_properties.keys())

#### 2.2 Define how to process which attributes

In [10]:
# Process based on patterns and keywords
pattern_attrs=['age','ethnicity', 'religion', 'educationlevel', 
               'causeofdeath', 'pastconviction', 'birthplace', 'residence']

# Process based on coreference
coref_attrs=['gender']

#### 2.3 Prepare the lexicons per attribute

##### 2.3.1 Prepare pattern keywords

In [11]:
pattern_data=defaultdict(dict)
for k,v in students_resources.items():
    new_v=utils.remove_stopwords(v)
    pattern_data[k.lower()]=new_v

#pattern_data['occupation']={o:o for o in emiel_resources['occupation-or-social-group']}

#pattern_data['ethnicity'] |= set(emiel_resources['ethnicity'])
#pattern_data['ethnicity'] |= set({'black', 'latino', 'white', 'hispanic', 'asian', 'latina', 
#       'african american', 'filipino', 'african-american', 'latinos',
#      'palestinian', 'chinese-american', 'blacks', 'german-iranian'})


#religions={o:o for o in (emiel_resources['religion']+students_resources['Religion'])}

##### 2.3.2 Prepare patterns

In [12]:
# Define exact patterns per attribute
patterns={'age': [r'\d\d?-year-old', r', \d\d?']}

In [13]:
for attr, keyword_json in pattern_data.items(): 
    attr_patterns=set()
    keywords=set(keyword_json.keys())
    for keyword in keywords:
        attr_patterns.add(r'\b%s\b' % keyword)
    patterns[attr]=attr_patterns

### 3. Attribute value extractors

#### 3.1 Proximity based extractor

In [14]:
def attr_extractor_proximity(text, people_spans, coref_spans, sentence_offsets, patterns, a_dict=None):
    """Generic extractor that operates based on patterns."""
    
    extracted_pairs=defaultdict(list)
    for pattern in patterns:
        r=re.compile(pattern, re.IGNORECASE)
        values=r.finditer(text)
        for val_found in values:
            span=val_found.span()
            value=val_found.group()
            value=value.replace('-year-old', '').replace(',', '').strip()
            if a_dict and value in a_dict:
                value=a_dict[value]
            person, distance=utils.find_closest_person(span, 
                                                       people_spans, 
                                                       coref_spans, 
                                                       sentence_offsets,
                                                       min_dist=1000)
            if person:
                extracted_pairs[person].append(tuple([distance, value]))
    clean_pairs=utils.get_closest_value_per_person(extracted_pairs)
    return clean_pairs

#### 3.2 Coreference based extractor

In [15]:
def attr_extractor_coref(clusters, names, values_json, debug=False): #, text, people_spans):
    
    person_data=defaultdict(list)

    if not clusters:
        return person_data
    
    for c in clusters:
        mentions=utils.stringify_cluster_mentions(c.mentions)
        for person_name in names:
            if utils.lookup_person_in_list(person_name, mentions):
                for m in c.mentions:
                    for txt in [m.text, m.lemma_]:
                        if txt.lower() in values_json.keys():
                            person_data[person_name].append(values_json[txt.lower()])
                            
    clean_data={}
    for person_name, gs in person_data.items():
        c=Counter(gs).most_common(1)[0][0]
        clean_data[person_name]=c
    return clean_data

#### 3.3 Run extraction of all properties for a document

In [16]:
def extract_properties(names, full_text, nlp):
    
    doc = nlp(full_text)
    # run coreference
    people_spans, coref_spans, clusters=utils.get_coref_spans(names, full_text, doc)
    
    sentence_offsets=utils.get_sentence_offsets(doc)
    
    all_extracted={}
    for attribute in pattern_attrs:
        all_extracted[attribute]=attr_extractor_proximity(full_text, 
                                                           people_spans, 
                                                           coref_spans, 
                                                           sentence_offsets, 
                                                           a_dict=pattern_data[attribute],
                                                           patterns=patterns[attribute])
    
    all_extracted['gender']=attr_extractor_coref(clusters, names, genders)
    
    #combine extractors
    combined=utils.singularize_data(all_extracted)
            
    return combined

In [17]:
# Test stuff

txt='Hello Peter Boer, 26 was shot in church from Houston. The white police guy failed.'
names=['Peter Boer']
extract_properties(names, txt, nlp)

{'Peter Boer': {'age': '26',
  'ethnicity': 'white/caucascian',
  'religion': 'christian',
  'causeofdeath': 'intentional',
  'residence': 'texas'}}

### 4. Run GVDB extractors

In [18]:
def process_gvdb_data(the_file, limit=200):

    all_gold_rows=[]
    all_sys_rows=[]

    with open(the_file, 'r') as csvfile:
        rdr = csv.reader(csvfile, delimiter='\t', quotechar='"')
        header=next(rdr)
        
        for c, row in enumerate(rdr):
            if c==limit: break

            full_text=row[2]
            data=json.loads(row[3])
            part_info=utils.get_participant_info(data)

            names=set(part_info.keys())
            if not len(names): continue

            system_data = extract_properties(names, full_text, nlp)
            all_sys_rows.append(system_data)
            all_gold_rows.append(part_info)
            
            c+=1
    return all_sys_rows, all_gold_rows

In [19]:
#all_sys_rows, all_gold_rows = process_gvdb_data(gvdb_articles_file, limit=200)

### 5. Run SE extractors

In [20]:
def process_se_data(annotation_file, text_file, limit=200):

    annotations=utils.load_pickle(annotation_file)
    all_texts_json=utils.load_pickle(text_file)
    
    all_gold_rows=[]
    all_sys_rows=[]
        
    storable = {}
        
    cnt=0

    for doc_id, part_data in annotations.items():
        storable[doc_id]={}
        
        names=utils.get_names(part_data)
        if doc_id not in all_texts_json:
            text=utils.load_text_from_json('%s/%s.json' % (se_all_documents_path, doc_id))
        else:
            text_json=all_texts_json[doc_id]
            conll_data=text_json['content']
            text=utils.conll_to_text(conll_data)
        
        properties=extract_properties(names, text, nlp)
        
        all_sys_rows.append(properties)
             
        for part_id, a_part_info in part_data.items():
            if 'Name' not in a_part_info.keys() or not a_part_info['Name']: continue

            name=a_part_info['Name'].strip()

            these_properties={}
            if name in properties.keys():
                for k,v in properties[name].items():
                    these_properties[map_properties[k]]=v

            these_properties['Name']=name
            storable[doc_id][part_id]=these_properties

        part_info=utils.transform_part_info(part_data)
        all_gold_rows.append(part_info)
        
        cnt+=1
        if cnt==limit: break
        
    return all_sys_rows, all_gold_rows, storable

In [21]:
se_system, se_gold, data_to_store = process_se_data(se_annotation_file, 
                                                    se_partial_text_file, 
                                                    limit=10000)

african american ad74870db8920e6986b76133117e7a82
african american 65d7c38113863b1bc6cee2e0fa38dc6c
hispanic/latin a51f975867abd1f18661d858fabf9c3f
white/caucascian 4026e33b1c2033fb5433740895842102
african american 6bb0bddf01c6230df34d1ce9cd7869a6
hispanic/latin 8695354c9046509b209a5aa92f48c199
african american 6bb255cee652c51bb6d34444b1a76705
african american e95dc411de0c45356810076ce4e50890
african american 42ab147825ee27d8a354517061216a91
african american 2a7e35c5897effac0205abe1f82af96d


In [22]:
utils.count_per_attribute(se_gold)

defaultdict(int,
            {'residence': 404,
             'age': 740,
             'gender': 757,
             'educationlevel': 117,
             'causeofdeath': 489,
             'deathplace': 566,
             'deathdate': 566,
             'birthplace': 6,
             'pastconviction': 32,
             'religion': 17,
             'medicalcondition': 8,
             'ethnicity': 10})

### 5. Benchmark extractors

#### 5.1 Benchmark GVDB data

In [23]:
# gvdb_attributes=['age', 'race', 'gender']
# utils.benchmark_extractors(all_sys_rows, all_gold_rows, gvdb_attributes)

#### 5.2 Benchmark on SemEval data

In [24]:
if 'age' in se_attributes:
    se_system=utils.map_age_to_group(se_system)

In [25]:
utils.benchmark_extractors(se_system, se_gold, se_attributes, debug='ethnicity')

128 sys african american
129 sys african american
135 gold african american
135 gold african american
176 gold hispanic/latin
239 gold white/caucascian
270 gold african american
294 gold hispanic/latin
305 gold african american
335 gold african american
364 gold african american
404 gold african american
421 sys african american
defaultdict(<class 'int'>, {'age': 192, 'educationlevel': 23, 'residence': 96, 'gender': 103, 'causeofdeath': 84, 'religion': 2, 'pastconviction': 8}) defaultdict(<class 'int'>, {'pastconviction': 76, 'causeofdeath': 139, 'educationlevel': 14, 'residence': 154, 'age': 14, 'birthplace': 1, 'gender': 14, 'ethnicity': 3, 'religion': 2}) defaultdict(<class 'int'>, {'age': 548, 'residence': 308, 'gender': 654, 'causeofdeath': 405, 'educationlevel': 94, 'birthplace': 6, 'religion': 15, 'pastconviction': 24, 'ethnicity': 10})


({'age': 0.9320388349514563,
  'ethnicity': 0.0,
  'causeofdeath': 0.37668161434977576,
  'religion': 0.5,
  'educationlevel': 0.6216216216216216,
  'pastconviction': 0.09523809523809523,
  'residence': 0.384,
  'birthplace': 0.0,
  'gender': 0.8803418803418803},
 {'age': 0.2594594594594595,
  'ethnicity': 0.0,
  'causeofdeath': 0.17177914110429449,
  'religion': 0.11764705882352941,
  'educationlevel': 0.19658119658119658,
  'pastconviction': 0.25,
  'residence': 0.2376237623762376,
  'birthplace': 0.0,
  'gender': 0.13606340819022458},
 {'age': 0.4059196617336152,
  'ethnicity': 0.0,
  'causeofdeath': 0.23595505617977527,
  'religion': 0.19047619047619047,
  'educationlevel': 0.29870129870129875,
  'pastconviction': 0.13793103448275862,
  'residence': 0.29357798165137616,
  'birthplace': 0.0,
  'gender': 0.2356979405034325})

### 6. Prepare SE output to be run by the baselines

Desired format: pickle

```
{
    doc_id:
    {
        part_id:
        {
            prop: value, 
            prop2: value2,
            ...
        }
    }
}
```

#### 6.1 Process the original data

In [26]:
utils.dump_pickle(data_to_store, se_output_file)

#### 6.2 Process the altered versions

In [27]:
for f in glob.glob('%s/*.p' % se_annotation_dir):
    if f.strip()!=se_annotation_file:
        print(f, se_annotation_file)
        altered_data=utils.load_pickle(f)
        
        not_found=0
        new_data=defaultdict(dict)
        for doc_id, doc_data in altered_data.items():
            doc_property_data=data_to_store[doc_id]
            
            for part_id, part_data in doc_data.items():
                if part_id not in doc_property_data.keys():
                    #print('NOT FOUND IN THE EXTRACTED DATA. document:', doc_id, '; participant:', part_id)
                    not_found+=1
                    continue
                    
                new_part_data={}
                if 'Name' in part_data:
                    new_part_data['Name'] = part_data['Name']
                for k, v in doc_property_data[part_id].items():
                    if k!='Name':
                        new_part_data[k]=v
                new_data[doc_id][part_id]=new_part_data

        print('not found number:', not_found)
        new_file_path='%s/%s' % (se_output_dir, f.split('/')[-1])
        print(new_file_path)
        
        utils.dump_pickle(new_data, new_file_path)

../data/input/partial/annotation/participants_samefirstname.p ../data/input/partial/annotation/participants_input.p
not found number: 9
extracted_data/partial/participants_samefirstname.p
../data/input/partial/annotation/participants_samename.p ../data/input/partial/annotation/participants_input.p
not found number: 9
extracted_data/partial/participants_samename.p
../data/input/partial/annotation/participants_samelastname.p ../data/input/partial/annotation/participants_input.p
not found number: 9
extracted_data/partial/participants_samelastname.p
